## Modeling with a Dataframe

After webscraping all of the Euros data and formatting into a dataframe then csv in my [euros_scraping.ipynb](http://localhost:8888/files/Documents/Coding/Notebooks/repos/euros-WebScraping/euros_scraping.ipynb?_xsrf=2%7C8c3fecd8%7C02d37b9b4c7c7af0dd7bca94af94aa7f%7C1716421226) notebook, I will now attempt to perform Modeling and predictions

THe hope is I scraped the relevant data, having all results recorded from Euro's and Euro's qualifiers into the csv.

In this notebook, I will use pandas as the main method of modeling this data.

In [276]:
import pandas as pd

In [277]:
matches = pd.read_csv("euros.csv", index_col=0)
matches["mDate"] = pd.to_datetime(matches["mDate"], format="%d.%m.%y")
matches.loc[matches["mDate"].dt.year > 2024, "mDate"] -= pd.DateOffset(years=100) 

# necesssary to offset pre-2000 mDates being interpreted as 20%yy (10.05.59 as 1959)

In [278]:
matches["hTeam"].value_counts().head()

hTeam
Italy       84
Germany     84
Spain       83
Denmark     81
Portugal    80
Name: count, dtype: int64

In [279]:
matches["aTeam"].value_counts().head()

aTeam
Spain          88
Italy          79
Netherlands    77
Denmark        76
Portugal       76
Name: count, dtype: int64

### Next Steps 

Currently, a single National team's data is split between home and away sections. For example, when we scraped the page data, sometimes Italy was listed as the left team and sometimes it was listed as the right team.

For brevity, I selected to set Italy as the home team whenever it appeared on the left and designated it as the away team when it appeared on the right.

It will be better to reorganize the data, coalescing all of Italy's matches together, re-stiching the data from

> hTeam --> team <BR>
> hGoals --> gScored <BR>
> hPenalties --> pScored <BR>
> hResult --> result <BR>
> aTeam --> opponent <BR>
> aGoals --> gAllowed <BR>
> aPenalties --> pAllowed <BR>

This will be difficult and require duplicate row data <br>
(i.e Italy as home team, Denmark as away, recording same match data for both in their own "team" as opposite's "opponent" so we can focus on a Nation's individual performance)

Instead of manipulating our current dataframe, it may be best to update into a new dataframe

In [280]:
# comp | mType | team | opponent | result | goals | gAllowed | penalties | pAllowed | mDate
mInfo = {"team": ["filler"], "opponent": ["fller"],
        "result": ['f'], "goals": [1.1], "gAllowed": [1.1], 
        "penalties": [1.1], "pAllowed": [1.1], "mDate": [pd.NA], 
        'mType': ["filler"], 'comp': ["filler"],}
cMatches = pd.DataFrame(data=mInfo)
cMatches[["mDate"]] = cMatches[["mDate"]].astype("datetime64[ns]")
cMatches = cMatches.drop(index=0)
cMatches

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp


In [281]:
for index, row in matches.iterrows():
    cMatches.loc[-1] = [row["hTeam"], row["aTeam"], row["hResult"], row["hGoals"], row["aGoals"], row["hPenalties"], row["aPenalties"], row["mDate"], row["mType"], row["comp"], ]
    cMatches.index += 1
    cMatches.loc[-1] = [row["aTeam"], row["hTeam"], row["aResult"], row["aGoals"], row["hGoals"], row["aPenalties"], row["hPenalties"], row["mDate"], row["mType"], row["comp"]]
    cMatches.index += 1
cMatches.index = list(range(len(cMatches["team"])))

In [282]:
cMatches.head()

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp
0,Georgia,Greece,W,0,0,4.0,2.0,2024-03-26,Final,"Euros 2024, Qualifiers"
1,Greece,Georgia,L,0,0,2.0,4.0,2024-03-26,Final,"Euros 2024, Qualifiers"
2,Wales,Poland,L,0,0,4.0,5.0,2024-03-26,Final,"Euros 2024, Qualifiers"
3,Poland,Wales,W,0,0,5.0,4.0,2024-03-26,Final,"Euros 2024, Qualifiers"
4,Ukraine,Iceland,W,2,1,NaN,NaN,2024-03-26,Final,"Euros 2024, Qualifiers"


In [283]:
cMatches.tail()

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp
6089,Denmark,ČSSR,L,1,5,NaN,NaN,1959-10-18,Round of 16,Euros 1960
6090,Ireland,ČSSR,W,2,0,NaN,NaN,1959-04-05,Qualifying Round,Euros 1960
6091,ČSSR,Ireland,L,0,2,NaN,NaN,1959-04-05,Qualifying Round,Euros 1960
6092,ČSSR,Ireland,W,4,0,NaN,NaN,1959-05-10,Qualifying Round,Euros 1960
6093,Ireland,ČSSR,L,0,4,NaN,NaN,1959-05-10,Qualifying Round,Euros 1960


In [284]:
cMatches["team"].value_counts().head()

team
Spain          171
Italy          163
Denmark        157
Portugal       156
Netherlands    156
Name: count, dtype: int64

## Setting category predictors

Below, we are adding additional columns as metrics for predicting a match. A team may perform differently based on the match type of the competition and their competitor 

The target column is if a team achieved its desired result, 

> a "W" was desired, equating to a 1 <br>
> a "L" or "D" was not desired, equating to a 0

In [285]:
cMatches["opponent"] = cMatches["opponent"].astype("category")
cMatches["mType"] = cMatches["mType"].astype("category")

dO_keys = cMatches["opponent"].to_list()
dM_keys = cMatches["mType"].to_list()

dO_values = cMatches["opponent"].cat.codes
dM_values = cMatches["mType"].cat.codes

d_Opp = dict(zip(dO_keys, dO_values))
d_Matches = dict(zip(dM_keys, dM_values))

cMatches["oppCode"] = cMatches["opponent"].cat.codes
cMatches["mCode"] = cMatches["mType"].cat.codes

In [286]:
points = {"W": 3, "D": 1, "L": 0} 
pointsR = {3: "W", 1: "D", 0: "L"}

cMatches["target"] = [ points[row["result"]] for index, row in cMatches.iterrows()]
cMatches.loc[cMatches["result"] == "D"]

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp,oppCode,mCode,target
22,Georgia,Norway,D,1,1,NaN,NaN,2023-03-28,Group Stage,"Euros 2024, Qualifiers",40,2,1
23,Norway,Georgia,D,1,1,NaN,NaN,2023-03-28,Group Stage,"Euros 2024, Qualifiers",19,2,1
50,Georgia,Scotland,D,2,2,NaN,NaN,2023-11-16,Group Stage,"Euros 2024, Qualifiers",46,2,1
51,Scotland,Georgia,D,2,2,NaN,NaN,2023-11-16,Group Stage,"Euros 2024, Qualifiers",19,2,1
56,Scotland,Norway,D,3,3,NaN,NaN,2023-11-19,Group Stage,"Euros 2024, Qualifiers",40,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6065,France,Greece,D,1,1,NaN,NaN,1958-12-03,Round of 16,Euros 1960,22,8,1
6076,Bulgaria,Yugoslavia,D,1,1,NaN,NaN,1959-10-25,Round of 16,Euros 1960,57,8,1
6077,Yugoslavia,Bulgaria,D,1,1,NaN,NaN,1959-10-25,Round of 16,Euros 1960,8,8,1
6086,Denmark,ČSSR,D,2,2,NaN,NaN,1959-09-23,Round of 16,Euros 1960,58,8,1


## Predictions 
With a RandomForestClassifer, we can run simulations with our category codes non-linearity,

We then split from a "trained" and "test" dataframe 

> A Train dataframe to . . . train our rf model on our selected predictors (opponent and match type) to see what the expected result (target) would be in our test model <br>
> A Test data frame to see how our predictions actually compare with that of the outcome 

In [287]:
from sklearn.ensemble import RandomForestClassifier

# associates non-linearity 
# i.e oppCode 22 doesn't imply numerical difference between oppCode 23
# just categorical differences

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

train_set = cMatches[cMatches["mDate"] < "2000-01-01"] 
test_set = cMatches[cMatches["mDate"] > "2000-01-01"]

predictors = ["oppCode", "mCode"]

rf.fit(train_set[predictors], train_set["target"])
# runs our forest model on known oppCode mCode combos, then we provide the result that came out of it "target"

preds = rf.predict(test_set[predictors])
# then with our test set, we produce a precition column on what result would be given oppCode and mCode

## Comparisons

With our trained rf, we then create predictions based on the same metrics for the test data set

Using the accuracy_score, we're able to see how often our trained model guessed correctlty

In [288]:
from sklearn.metrics import accuracy_score

# measures our predictions 
acc = accuracy_score(test_set["target"], preds)
acc

# compares the likeness of each prediction to the actual result (target) and reports our accuracy

0.5438496583143508

In [289]:
combined = pd.DataFrame(dict(actual=test_set["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1,3
actual,,,
0,939,5,503
1,330,4,284
3,472,8,967


In [290]:
from sklearn.metrics import precision_score

"""
Precision Score then defined as 

    ps = tp / (tp + fp) 

    tp = true positive 
    fp = false positive 

Essentially reports the score "how often did we correctly call a winning result
"""

precision_score(test_set["target"], preds, average=None)

array([0.5393452 , 0.23529412, 0.55131129])

## Predictions 

With our model, we will now predict the results of Group Stage Fixtures with our training model and move on from there until a winner is decided

Here's what happening below 

1. Web-scraping for all the Group Stages in the Euros 2024
2. Isolating to just the fiixtures
3. Cleaning up given fixture table into our new tables
4. Repeating the data twice per table (with swapped team and opponent)

Now en_tables should be an array of our tables cleaned up with doubled match data


In [291]:
import requests
import io 
from bs4 import BeautifulSoup

e_2024 = "https://terrikon.com/en/euro-2024"
e_2024_page = requests.get(e_2024)
e_html = e_2024_page.text

e_tables = pd.read_html(io.StringIO(e_html))
group_tables = [table for table in e_tables if table.shape[0] != 6]
group_tables = [table.drop(["Unnamed: 0", "G", "S", "M"], axis=1) for table in group_tables]
group_tables = [table.rename(columns={"Unnamed: 1": "Team"}) for table in group_tables]
lineup_tables = [table for table in e_tables if table.shape[0] == 6]
en_tables = [] 

mInfo = {"team": ["filler"], "opponent": ["fller"],  "mDate": [pd.NA], 
        'mType': ["filler"], 'comp': ["filler"], "oppCode": ["filler"], "mCode": ["filler"]}
for table in lineup_tables:
    f_table = pd.DataFrame(data=mInfo)
    f_table = f_table.drop(index=0)
    table[5] = pd.to_datetime(matches["mDate"], format="%d.%m.%y %h:%m")
    for index, row in table.iterrows():
        f_table.loc[-1] = [row[1], row[3], row[5], "Group Stage", "Euros 2024", d_Opp[row[3]], d_Matches["Group Stage"]]
        f_table.index += 1
        f_table.loc[-1] = [row[3], row[1], row[5], "Group Stage", "Euros 2024", d_Opp[row[1]], d_Matches["Group Stage"]]
        f_table.index += 1
    en_tables.append(f_table)


## Predicting Group Stage 

Per table, we are now going to produce a prediction column to see who the result of the match.

Should the same-matchup produce different results, we will chalk it up as a tie



In [292]:
group_tables[0]

,Team,W,D,L,-,Pts
0,Switzerland,0,0,0,-,0
1,Hungary,0,0,0,-,0
2,Scotland,0,0,0,-,0
3,Germany,0,0,0,-,0


In [293]:
en_tables[0]

,team,opponent,mDate,mType,comp,oppCode,mCode
11,Germany,Scotland,2024-03-26,Group Stage,Euros 2024,46,2
10,Scotland,Germany,2024-03-26,Group Stage,Euros 2024,20,2
9,Hungary,Switzerland,2024-03-26,Group Stage,Euros 2024,52,2
8,Switzerland,Hungary,2024-03-26,Group Stage,Euros 2024,23,2
7,Germany,Hungary,2024-03-26,Group Stage,Euros 2024,23,2
6,Hungary,Germany,2024-03-26,Group Stage,Euros 2024,20,2
5,Scotland,Switzerland,2024-03-21,Group Stage,Euros 2024,52,2
4,Switzerland,Scotland,2024-03-21,Group Stage,Euros 2024,46,2
3,Switzerland,Germany,2024-03-21,Group Stage,Euros 2024,20,2
2,Germany,Switzerland,2024-03-21,Group Stage,Euros 2024,52,2


### Making a prediction 

In [294]:
def run_prediction(uMatches, kMatches, metrics): 
    known_set = kMatches[cMatches["mDate"] > "2015-01-01"] 
    forest = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1) 

    forest.fit(kMatches[metrics], kMatches["target"])

    kPreds = forest.predict(known_set[metrics])
    uPreds = forest.predict(uMatches[metrics])

    ps = precision_score(known_set["target"], kPreds, average=None)
    
    return uPreds, ps 

In [295]:
for group in en_tables: 
    preds, confidence = run_prediction(group, cMatches, ["oppCode", "mCode"])
    group["prediction"] = [ pointsR[pred] for pred in preds]

In [296]:
en_tables[0]

,team,opponent,mDate,mType,comp,oppCode,mCode,prediction
11,Germany,Scotland,2024-03-26,Group Stage,Euros 2024,46,2,L
10,Scotland,Germany,2024-03-26,Group Stage,Euros 2024,20,2,L
9,Hungary,Switzerland,2024-03-26,Group Stage,Euros 2024,52,2,L
8,Switzerland,Hungary,2024-03-26,Group Stage,Euros 2024,23,2,L
7,Germany,Hungary,2024-03-26,Group Stage,Euros 2024,23,2,L
6,Hungary,Germany,2024-03-26,Group Stage,Euros 2024,20,2,L
5,Scotland,Switzerland,2024-03-21,Group Stage,Euros 2024,52,2,L
4,Switzerland,Scotland,2024-03-21,Group Stage,Euros 2024,46,2,L
3,Switzerland,Germany,2024-03-21,Group Stage,Euros 2024,20,2,L
2,Germany,Switzerland,2024-03-21,Group Stage,Euros 2024,52,2,L
